In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

print(os.getcwd())

In [ ]:
# Define speeds for 0-40 Hz
speeds = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))



In [ ]:
class CrossWireDataset(Dataset):
    def __init__(self, magFile, angFile, readingsFile, transform=None, target_transform=None):
        
        # Construct the labels
        tmpMag = pd.read_csv(magFile)
        tmpAng = pd.read_csv(angFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.angs = torch.Tensor(tmpAng.to_numpy())
        
        # Construct the features and place them into readings (X). 
        tmpReadings = pd.read_csv(readingsFile)
        tmpReadings = tmpReadings.to_numpy()
        print(tmpReadings.shape)
        LL = tmpReadings.shape[0]
        tmpReadings2 = np.zeros((LL, 3))
        for k in range(LL):
            tmpReadings2[k, 0] = np.max(np.abs(tmpReadings[k, :]))
            tmpReadings2[k, 1] = np.argmax(np.abs(tmpReadings[k, :]))
            tt = int(tmpReadings2[k,1])
            tmpReadings2[k, 2] = np.abs(tmpReadings[k, (tt-1)%6])/(np.abs(tmpReadings[k, (tt+1)%6]) + 0.05)
        self.readings = torch.Tensor(tmpReadings2)
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        ang = self.angs[idx]
        label = torch.cat((mag, ang), 0)
        
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            mag = self.target_transform(mag)

        return reading, label

In [ ]:
class WindMagDataset(Dataset):
    def __init__(self, magFile, readingsFile, transform=None, target_transform=None):
        tmpMag = pd.read_csv(magFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            mag = self.target_transform(mag)

        return reading, mag

class WindAngDataset(Dataset):
    def __init__(self, angFile, readingsFile, transform=None, target_transform=None):
        tmpAng = pd.read_csv(angFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.angs = torch.Tensor(tmpAng.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.angs)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        ang = self.angs[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            ang = self.target_transform(ang)

        return reading, ang

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, crosswire=False, fullAngles=False):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        
        if crosswire:
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(3, 25),
                nn.ReLU(),
                nn.Linear(25, 15),
                nn.ReLU(),
                nn.Linear(15, 2),
            )
        else:
            if fullAngles:
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(6, 20),
                    nn.ReLU(),
                    nn.Linear(20, 8),
                    nn.ReLU(),
                    nn.Linear(8, 1),
                )
            else:
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(6, 10),
                    nn.ReLU(),
                    nn.Linear(10, 5),
                    nn.ReLU(),
                    nn.Linear(5, 1),
                )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

In [ ]:
# Define miscellaneous learning quantities
learning_rate = 1e-3
batch_size = 64
epochs = 8
loss_fn = nn.MSELoss()


In [ ]:
# Make dataset
# Change this as desired in {1, 2, 3, 4, 5}
dataSetType = 5

# Don't change these; the if statements take care of them
# Set network parameters in NeuralNetwork class
fullAnglesVal = False
crosswireVal = False
epochs = 5

if dataSetType==1:
    trainY = 'MagTrain/mags.csv'
    trainX = 'MagTrain/readings.csv'
    testY = 'MagTest/mags.csv'
    testX = 'MagTest/readings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    
elif dataSetType==2:
    trainY = 'MagTrain/angsrad.csv'
    trainX = 'MagTrain/readings.csv'
    testY = 'MagTest/angsrad.csv'
    testX = 'MagTest/readings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    
elif dataSetType==3:
    trainY1 = 'CrossTrain/crossmags.csv'
    trainY2 = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY1 = 'CrossTest/crossmags.csv'
    testY2 = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = CrossWireDataset(trainY1, trainY2, trainX, transform=None)
    testing_data = CrossWireDataset(testY1, testY2, testX, transform=None)
    
    crosswireVal = True
    epochs = 10
    
elif dataSetType==4:
    
    trainY = 'CrossTrain/crossmags.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossmags.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    
elif dataSetType==5:
    trainY = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    
    fullAnglesVal = True
    
else:
    raise ValueError('Not a valid dataSetType index (must be in {1, 2, 3, 4, or 5})')
    
# Make training and testing data
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)
model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, opt)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")